#Data Collection for Predicting Real Estate Prices in Warsaw

## Install and import libs

In [80]:
!pip install requests beautifulsoup4 pandas

In [84]:
from bs4 import BeautifulSoup
import requests
import csv
import time
import pandas as pd
import re
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## otodom.pl scrapper

### Get data from otodom.pl

In [114]:
# Parameter specifying the maximum number of pages to be processed
max_pages = 100

# Parameter specifying the number of start page
start_page = 56

# URL for Warsaw
otodom_url = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?page='

# HTTP headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# List to store records
records_otodom = []

def scrap_apartment_details(url):
    """
    This function scrapes detailed data from a specific apartment listing.

    - It sends an HTTP request to the provided URL to fetch the HTML content.
    - Parses the HTML using BeautifulSoup to extract various details.
    - Extracted details include price, price per square meter, number of rooms, area, location, description, update date, post date, and additional information.

    The function handles missing values by checking if elements exist before attempting to extract text.
    If an element is not found, it assigns 'N/A' as a placeholder.
    Returns a list of extracted details for further use.

    Args:
        url (str): The URL of the apartment listing.

    Returns:
        list: A list containing the extracted apartment details:
            - price
            - price per square meter
            - number of rooms
            - area
            - location
            - description
            - updated date
            - posted date
            - additional information 1
            - additional information 2
    """
    c = requests.get(url, headers=headers)
    soup_content = BeautifulSoup(c.content, 'html.parser')

    price = soup_content.find('strong', {'aria-label': 'Cena'}).text.strip() if soup_content.find('strong', {'aria-label': 'Cena'}) else 'N/A'
    price_per_m2 = soup_content.find('div', {'class': 'css-h8msn7 e1w5xgvx4'}).text.strip() if soup_content.find('div', {'class': 'css-h8msn7 e1w5xgvx4'}) else 'N/A'
    rooms = soup_content.find('div', {'aria-label': 'Liczba pokoi'}).text.strip() if soup_content.find('div', {'aria-label': 'Liczba pokoi'}) else 'N/A'
    area = soup_content.find('div', {'aria-label': 'Powierzchnia'}).text.strip() if soup_content.find('div', {'aria-label': 'Powierzchnia'}) else 'N/A'
    location = soup_content.find('a', {'class': 'css-1jjm9oe e42rcgs1'}).text.strip() if soup_content.find('a', {'class': 'css-1jjm9oe e42rcgs1'}) else 'N/A'
    description = soup_content.find('div', {'data-cy': 'adPageAdDescription'}).text.strip() if soup_content.find('div', {'data-cy': 'adPageAdDescription'}) else 'N/A'
    update_info = soup_content.find_all('p', {'class': 'e82kd4s2 css-htq2ld'})

    if len(update_info) >= 2:
        updated_date = update_info[0].text.strip()
        posted_date = update_info[1].text.strip()
    else:
        updated_date = posted_date = 'N/A'

    additional_info_1 = soup_content.find_all('div', {'class': 'css-1xbf5wd etn78ea0'})
    additional_info_1_text = ', '.join([info.text.strip() for info in additional_info_1]) if additional_info_1 else 'brak informacji'

    additional_info_2 = soup_content.find_all('div', {'class': 'etn78ea4 css-1nh5rfu'})
    additional_info_2_text = ', '.join([info.text.strip() for info in additional_info_2]) if additional_info_2 else 'brak informacji'

    return [price, price_per_m2, rooms, area, location, description, updated_date, posted_date, additional_info_1_text, additional_info_2_text]

def scrap_otodom(url):
    """
    Scrapes the Otodom listing page for apartment urls, then scrapes detailed information
    from each individual apartment listing.

    Args:
        url (str): The URL of the Otodom listing page.

    Returns:
        None: The function appends scraped data (source, link, title, and detailed info) to the global list `records_otodom`.
    """
    c = requests.get(url, headers=headers)
    soup_content = BeautifulSoup(c.content, 'html.parser')
    base_url = 'https://www.otodom.pl'

    # Search for all urls on page
    for listing in soup_content.find_all('a', {'data-cy': 'listing-item-link'}):
        relative_link = listing.get('href')
        full_link = base_url + relative_link

        title = listing.find('p', {'data-cy': 'listing-item-title'}).text.strip() if listing.find('p', {'data-cy': 'listing-item-title'}) else 'N/A'

        details = scrap_apartment_details(full_link)

        row = ["Otodom", full_link, title] + details
        records_otodom.append(row)
        print("Scraped:", full_link)

def save_to_csv(filename, records):
    """
    Saves the scraped records to a CSV file with specified column headers.

    Args:
        filename (str): The name of the CSV file where the data will be saved.
        records (list): A list of records, where each record is a list of scraped data.

    Returns:
        None: The function writes the data to the specified CSV file.
    """
    try:
        with open(filename, "w", newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            # Dodanie nagłówków CSV
            writer.writerow(['Source', 'Url', 'Title', 'Price', 'Pms', 'Rooms', 'Area', 'Location', 'Desc', 'Update date', 'Creation date' ,'Additional information 1', 'Additional information 2'])
            for record in records:
                writer.writerow(record)
        print(f'SaveToFile: {filename} OK.')
    except Exception as e:
        print(f'SaveToFile: ERROR. {e}')

def loop_for_scrap(base_url,start_page, max_pages):
    """
    Loops through the specified number of pages, sending a request to each page,
    and scrapes apartment listings using the scrap_otodom function.

    Args:
        base_url (str): The base URL of the listings with a page placeholder.
        max_pages (int): The maximum number of pages to scrape.
        start_page (int): The page number to start scraping from.

    Returns:
        None: Scrapes data from each page and appends the results to the global list.
    """
    for page_number in range(start_page, max_pages + 1):
        url = base_url + str(page_number)
        response = requests.head(url, headers=headers)
        if response.status_code < 400:
            scrap_otodom(url)
            time.sleep(2)
        else:
            break
        time.sleep(5)  # Delay between requests
        print(f"Page {page_number}")

# Start sraping
print("Starting scrap for Otodom...")
loop_for_scrap(otodom_url, start_page, max_pages)

# Global value for file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save data to .csv
save_to_csv(f'/content/drive/My Drive/Projects/Predicting Real Estate Prices in Warsaw/raw_otodom_{timestamp}.csv', records_otodom)


Starting scrap for Otodom...
Scraped: https://www.otodom.pl/pl/oferta/3-pokojowe-obok-saska-kepa-super-widok-na-centrum-ID4rVur
Scraped: https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-42m2-balkon-bez-prowizji-ID4psCu
Scraped: https://www.otodom.pl/pl/oferta/ochota-4-pok-76-66-m-garaz-ID4rUpF
Scraped: https://www.otodom.pl/pl/oferta/narozne-3-sypialnie-duzy-balkon-przestronne-ID4rdMh
Scraped: https://www.otodom.pl/pl/oferta/klimatyczne-2-pok-wsrod-zieleni-na-mokotowie-ID4saNZ
Scraped: https://www.otodom.pl/pl/oferta/praga-poludnie-2-pokoje-1-pietro-paca-siennicka-ID4s2DR
Scraped: https://www.otodom.pl/pl/oferta/oddzielna-kuchnia-garaz-komorka-lokatorska-ID4r1yg
Scraped: https://www.otodom.pl/pl/oferta/przestronne-studio-z-mozliwoscia-podzialu-na-2-pok-ID4rp8P
Scraped: https://www.otodom.pl/pl/oferta/mieszkanie-w-kamienicy-z-winda-w-sercu-mokotowa-ID4rsTM
Scraped: https://www.otodom.pl/pl/oferta/przestronne-studio-z-mozliwoscia-podzialu-na-2-pok-ID4rFJ5
Scraped: https://www.otodom

In [115]:
df = pd.read_csv(f'/content/drive/My Drive/Projects/Predicting Real Estate Prices in Warsaw/raw_otodom_{timestamp}.csv')

### Data cleaning

In [116]:
def replace_strings(column_data, remove_text, new_value):
    """
    This function replaces a specific substring in a given column value with a new value.

    Arguments:
      column_data (str): The content of a single cell in a column.
      remove_text (str): The substring that should be removed or replaced.
      new_value (str): The value that will replace the specified substring.
    """
    if pd.isna(column_data):  #Check NaN
        return column_data

    # Replace string
    data = column_data.replace(remove_text, new_value)

    return data

def extract_area_from_column(df, column_name, new_column_name):
    """
    Extracts the area (in m²) from the beginning of a text in a specified column and stores it in a new column, while removing the area from the original column.

    Arguments:
        df (DataFrame): The DataFrame containing the data.
        column_name (str): The column to extract the area value from.
        new_column_name (str): The column where the extracted area value will be stored.
    """
    area_regex = r'^(\d+(\.\d+)?)m²'

    df[new_column_name] = df[column_name].apply(lambda x: re.search(area_regex, x).group(1) if isinstance(x, str) and re.search(area_regex, x) else None)

    df[column_name] = df[column_name].apply(lambda x: re.sub(area_regex, '', x).strip() if isinstance(x, str) else x)

    return df


def extract_rooms_from_column(df, column_name, new_column_name):
    """
    Extracts the number of rooms from a specified column and stores it in a new column, while removing the room count from the original column.

    Arguments:
        df (DataFrame): The DataFrame containing the data.
        column_name (str): The column to extract the number of rooms from.
        new_column_name (str): The column where the extracted room count will be stored.
    """
    rooms_regex = r'^(\d+)\s*(pokój|pokoje)'

    df[new_column_name] = df[column_name].apply(lambda x: re.search(rooms_regex, x).group(1) if isinstance(x, str) and re.search(rooms_regex, x) else None)

    df[column_name] = df[column_name].apply(lambda x: re.sub(rooms_regex, '', x).strip() if isinstance(x, str) else x)

    return df

def extract_key_value_pairs(column_data):
    """
    Extracts key-value pairs from a string where the key is followed by a colon and the value comes after.

    Arguments:
        column_data (str): The text containing key-value pairs to extract.
    """
    key_value_regex = r'([A-Z][a-zA-Z\s]*):([^A-Z]*)'

    pairs = []

    if isinstance(column_data, str):
        matches = re.findall(key_value_regex, column_data)
        for match in matches:
            key = match[0].strip()
            value = match[1].strip()
            pairs.append((key, value))

    return pairs

def extract_key_value_columns(df, column_name):
    """
    Extracts key-value pairs from a specified column and creates new columns for each key, with corresponding values.

    Arguments:
        df (DataFrame): The DataFrame containing the data.
        column_name (str): The column from which key-value pairs will be extracted.
    """
    key_value_dict = {}

    for index, row in df.iterrows():
        key_value_pairs = extract_key_value_pairs(row[column_name])

        for key, value in key_value_pairs:
            if key not in key_value_dict:
                key_value_dict[key] = [None] * len(df)
            key_value_dict[key][index] = value

    for key, values in key_value_dict.items():
        df[key] = values

    return df

def extract_location_info(row):
    """
    Analyzes the 'Location' column to extract city, district, voivodeship, and street information.

    Arguments:
        row (Series): A row from the DataFrame containing location data.

    Returns:
        Series: The modified row with new fields for city, district, voivodeship, and street.
    """
    location = row['Location']
    if not isinstance(location, str):
        return row

    city, district, voivodeship, street = None, None, None, None

    parts = location.split(',')

    if "mazowieckie" in location:
        voivodeship = "mazowieckie"

    if "Warszawa" in location:
        city = "Warszawa"

    for part in parts:
        part = part.strip()
        if part in districts:
            district = part
            break

    street_match = re.search(r'(ul\.[^,]*),', location)
    if street_match:
        street = street_match.group(1).strip()

    row['City'] = city
    row['District'] = district
    row['Voivodeship'] = voivodeship
    row['Street'] = street

    return row

# Delete "Mieszkanie na sprzedaż" and "Budynek i materiały" strings
df['Additional information 1'] = df['Additional information 1'].apply(lambda x: replace_strings(x, "Mieszkanie na sprzedaż", ''))
df['Additional information 2'] = df['Additional information 2'].apply(lambda x: replace_strings(x, "Budynek i materiały",''))

# Extract area
df = extract_area_from_column(df, 'Additional information 1', 'Area')

# Extract rooms
df = extract_rooms_from_column(df, 'Additional information 1', 'Rooms')


# Apply function to "Additional information 1" and "Additional information 2" columns
df = extract_key_value_columns(df, 'Additional information 1')
df = extract_key_value_columns(df, 'Additional information 2')

# Delete "Aktualizacja: " and "Dodano: "
df['Update date'] = df['Update date'].apply(lambda x: replace_strings(x, "Aktualizacja: ", ''))
df['Creation date'] = df['Creation date'].apply(lambda x: replace_strings(x, "Dodano: ", ''))

# Dictionary with polish words and
column_name_mapping = {
    'Ogrzewanie': 'Heating',
    'Czynsz': 'Rent',
    'Rynek': 'Market',
    'Informacje dodatkowe': 'Additional Information',
    'Rok budowy': 'Year of construction',
    'Winda': 'Elevator',
    'Rodzaj zabudowy': 'Building type',
    'Zabezpieczenia': 'Security',
    'Media': 'Utilities',
    'Okna': 'Windows',
    'Certyfikat energetyczny': 'Energy certificate'
}

# Rename columns using mapping
df.rename(columns=column_name_mapping, inplace=True)

districts = [
    "Bemowo", "Białołęka", "Bielany", "Mokotów", "Ochota", "Praga-Południe",
    "Praga-Północ", "Rembertów", "Śródmieście", "Targówek", "Ursus", "Ursynów",
    "Wawer", "Wesoła", "Wilanów", "Włochy", "Wola", "Żoliborz"
]

#Apply extract_location_info
df = df.apply(extract_location_info, axis=1)

# Replace strings in Security
df['Security'] = df['Security'].apply(lambda x: replace_strings(x, " / ", ','))


# Save data to .csv file
df.to_csv(f'/content/drive/My Drive/Projects/Predicting Real Estate Prices in Warsaw/clean_otodom_{timestamp}.csv', index=False)
df


,Additional Information,Additional information 1,Additional information 2,Area,Building type,City,Creation date,Desc,District,Elevator,...,Security,Source,Street,Title,Update date,Url,Utilities,Voivodeship,Windows,Year of construction
0,balkon piwnica oddzielna kuchnia,Ogrzewanie:miejskiePiętro:9/10Czynsz:750 złSta...,Rok budowy:1980Winda:takRodzaj zabudowy:blokOk...,48,blok,Warszawa,8.08.2024,Do sprzedania bezpośrednio od właściciela 3 po...,Praga-Południe,tak,...,"drzwi,okna antywłamaniowe domofon,wideofon",Otodom,None,"3 pokojowe obok Saska Kępa, Super widok na Cen...",16.09.2024,https://www.otodom.pl/pl/oferta/3-pokojowe-obo...,telewizja kablowa,mazowieckie,plastikowe,1980
1,balkon garaż/miejsce parkingowe,Ogrzewanie:brak informacjiPiętro:1/3Czynsz:bra...,NaN,42.73,None,Warszawa,4.03.2024,2-pokojowe mieszkanie numer 20 na 1. piętrze ...,Białołęka,None,...,None,Otodom,ul. Duninów,2-pokojowe mieszkanie 42m2 + balkon Bez Prowizji,16.09.2024,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,None,mazowieckie,None,None
2,balkon garaż/miejsce parkingowe,Ogrzewanie:miejskiePiętro:3/5Czynsz:brak infor...,Winda:takRodzaj zabudowy:blokOkna:plastikoweBe...,76.66,blok,Warszawa,6.08.2024,"LEX NIERUCHOMOŚCI MARTA KOSMOWSKA, TEL. \nOfe...",Włochy,tak,...,"drzwi,okna antywłamaniowe domofon,wideofon",Otodom,None,"Ochota , 4 pok. 76,66 m, garaż",16.09.2024,https://www.otodom.pl/pl/oferta/ochota-4-pok-7...,telewizja kablowa internet,mazowieckie,plastikowe,None
3,balkon garaż/miejsce parkingowe,Ogrzewanie:miejskiePiętro:3/6Czynsz:850 złStan...,Rok budowy:2005Winda:takRodzaj zabudowy:blokMa...,75.32,blok,Warszawa,24.06.2024,"Oferuję na sprzedaż przestronne, jasne o funkc...",Bemowo,tak,...,"domofon,wideofon",Otodom,ul. Grodkowska,"Narożne,3 sypialnie,duży balkon, przestronne",15.09.2024,https://www.otodom.pl/pl/oferta/narozne-3-sypi...,None,mazowieckie,plastikowe,2005
4,balkon piwnica,Ogrzewanie:miejskiePiętro:1/4Czynsz:490 złStan...,Winda:nieRodzaj zabudowy:blokOkna:plastikoweCe...,48.5,blok,Warszawa,26.08.2024,"Oferta bezpośrednia. Klimatyczne, widne, ciche...",Mokotów,nie,...,"domofon,wideofon",Otodom,ul. Konstancińska 7,Klimatyczne 2 pok. wśród zieleni na Mokotowie,15.09.2024,https://www.otodom.pl/pl/oferta/klimatyczne-2-...,telewizja kablowa internet telefon,mazowieckie,plastikowe,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,None,brak informacji,brak informacji,None,None,NaN,NaN,NaN,NaN,None,...,None,Otodom,NaN,Penthouse/5 pokoi/Wilanów/bez prowizji,NaN,https://www.otodom.pl/pl/oferta/penthouse-5-po...,None,NaN,None,None
448,None,brak informacji,brak informacji,None,None,NaN,NaN,NaN,NaN,None,...,None,Otodom,NaN,2 pokoje/ przestronne/ Miasteczko Wilanów,NaN,https://www.otodom.pl/pl/oferta/2-pokoje-przes...,None,NaN,None,None
449,None,brak informacji,brak informacji,None,None,NaN,NaN,NaN,NaN,None,...,None,Otodom,NaN,wyjątkowa oferta/Kabaty/4 pokoje,NaN,https://www.otodom.pl/pl/oferta/wyjatkowa-ofer...,None,NaN,None,None
450,None,brak informacji,brak informacji,None,None,NaN,NaN,NaN,NaN,None,...,None,Otodom,NaN,całe piętro- 2 lokale/taras 185m2/ blisko metro,NaN,https://www.otodom.pl/pl/oferta/cale-pietro-2-...,None,NaN,None,None


## olx.pl API

## Additional data from description using LLM

## data on additional places in the area